In [20]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import h5py
import copy

import amici
from petab.C import *
import petab
import petab.C
import pypesto
import pypesto.petab
from pypesto.optimize import minimize
from pypesto.startpoint import uniform
from pypesto.engine import MultiProcessEngine, MultiThreadEngine
from pypesto.optimize.optimizer import FidesOptimizer
import pypesto.optimize as optimize

from pypesto.visualize import waterfall
from pypesto.visualize import parameters
from pypesto.visualize.model_fit import visualize_optimized_model_fit
from pypesto.visualize import profiles

import pypesto.profile as profile
from pypesto.optimize import ScipyOptimizer
from pypesto.profile.options import ProfileOptions

from pypesto.visualize.model_fit import visualize_optimized_model_fit, _get_simulation_rdatas

In [21]:
def hex_to_rgba_gradient(color1, color2, n):
    '''
    Create a gradient in rgba between two hex colors
    '''
    # Convert to rgba
    c1 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color1))
    c2 = matplotlib.colors.to_rgba(matplotlib.colors.hex2color(color2))

    return [[(c1[i]*(n-j-1) + c2[i]*j)/(n-1) for i in range(4)] for j in range(n)]

# find the index for cut off based on Chi square distribution CI 95%
def find_cut_off_index(result, ci = 0.95):
    '''
    Find the cut off index for the data based on the Chi square distribution
    '''

    # calculate the chi square distribution
    cut_off_value = scipy.stats.chi2.ppf(ci, 1)

    # find the index
    best_fval = result.optimize_result.list[0].fval

    for i in range(len(result.optimize_result.list)):
        if result.optimize_result.list[i].fval > best_fval + cut_off_value:
            break
    
    return i - 1

def find_cut_off_x_trace(result, ci = 0.95, flatten = True):

    cut_off_value = scipy.stats.chi2.ppf(ci, 1)
    best_fval = result.optimize_result.list[0].fval

    # store the optimized x trace that are below the cut off value
    x_trace_within_cut_off = []
    if flatten:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off.append(x_trace[j])
    else:
        for i in range(find_cut_off_index(result, ci)):
            
            fval_trace = result.optimize_result.list[i].history.get_fval_trace()
            x_trace = result.optimize_result.list[i].history.get_x_trace()

            x_trace_within_cut_off_i = []
            for j in range(len(fval_trace)):
                if fval_trace[j] < best_fval + cut_off_value:
                    x_trace_within_cut_off_i.append(x_trace[j])
            x_trace_within_cut_off.append(x_trace_within_cut_off_i)

    return x_trace_within_cut_off

In [22]:
# Plot setting
plt.rcParams['font.size'] = 30

dpi = 100
wid = int(2560/dpi)
hei = int(1600/dpi)

In [23]:
# number of optimization runs
n_runs = 5000

In [24]:
np.random.seed(500)

In [25]:
# optimization
hierarchical = False

petab_yaml = 'petab_files/dividing_infected_cells.yaml'
petab.validate(petab_yaml)
petab_problem = petab.Problem.from_yaml(petab_yaml)

problem = pypesto.petab.PetabImporter(
        petab_problem,
        hierarchical=hierarchical,
        model_name=f"DIVIDING_INFECTED_CELLS_Model",
    ).create_problem(force_compile=True)

problem.objective.amici_model.setAllStatesNonNegative()

# some model properties
print("Model parameters:", list(problem.objective.amici_model.getParameterIds()), "\n")
print("Model const parameters:", list(problem.objective.amici_model.getFixedParameterIds()), "\n")
print("Model outputs:   ", list(problem.objective.amici_model.getObservableIds()), "\n")
print("Model states:    ", list(problem.objective.amici_model.getStateIds()), "\n")

Compiling amici model to folder /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model.
2024-11-11 14:43:58.380 - amici.petab.sbml_import - INFO - Importing model ...
2024-11-11 14:43:58.381 - amici.petab.sbml_import - INFO - Validating PEtab problem ...
2024-11-11 14:43:58.423 - amici.petab.sbml_import - INFO - Model name is 'DIVIDING_INFECTED_CELLS_Model'.
Writing model code to '/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model'.
2024-11-11 14:43:58.425 - amici.petab.sbml_import - INFO - Species: 0
2024-11-11 14:43:58.425 - amici.petab.sbml_import - INFO - Global parameters: 17
2024-11-11 14:43:58.425 - amici.petab.sbml_import - INFO - Reactions: 0
2024-11-11 14:43:58.432 - amici.sbml_import - DEBUG - Finished validating SBML                    ++ (9.19E-04s)
2024-11-11 14:43:58.434 - amici.sbml_import - DEBUG - Finished converting SBML local parameters   ++ (2

running build_ext
running AmiciBuildCMakeExtension
------------------------------ model_ext ------------------------------

==> Configuring:
$ cmake -S /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model -B /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model/build_model_ext -G Ninja -DCMAKE_MAKE_PROGRAM=/Users/yuhongliu/Documents/venv_ov/bin/ninja -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX:PATH=/Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_models/0.27.0/DIVIDING_INFECTED_CELLS_Model/DIVIDING_INFECTED_CELLS_Model -DCMAKE_VERBOSE_MAKEFILE=ON -DCMAKE_PREFIX_PATH=/Users/yuhongliu/Documents/venv_ov/lib/python3.12/site-packages/amici -DAMICI_PYTHON_BUILD_EXT_ONLY=ON -DPython3_EXECUTABLE=/Users/yuhongliu/Documents/venv_ov/bin/python3

==> Building:
$ cmake --build /Users/yuhongliu/Documents/OV/models_refit_for_IFAC/hidden_model/amici_mod

# Gradient Check

In [26]:
startpoints = problem.get_startpoints(n_starts=2)

In [27]:
problem.objective.check_grad(
    x = startpoints[0],
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-4.441204e+03,-4415.069142,-4467.290064,-4441.179603,52.220923,0.024858,5.597168e-06
kappa,3.767917e+02,402.408181,349.550105,375.979143,52.858076,0.812580,2.161238e-03
psi,-6.221777e+01,-33.480002,-90.844546,-62.162274,57.364544,0.055496,8.927586e-04
phi,-2.889060e+02,-260.384722,-317.569465,-288.977094,57.184743,0.071057,2.458915e-04
beta,-2.977135e-09,28.598364,-28.598368,-0.000002,57.196732,0.000002,3.171587e-01
alpha,-5.439484e-03,28.577706,-28.616018,-0.019156,57.193724,0.013717,7.164229e-01
delta,6.030195e+00,34.874088,-22.599848,6.137120,57.473937,0.106925,1.742259e-02
sigma_add,-1.028216e+04,-10253.380133,-10310.933673,-10282.156903,57.553540,0.001284,1.249140e-07
sigma_mul,-1.967307e+03,-1938.724641,-1995.889305,-1967.306973,57.164664,0.000047,2.393937e-08


In [28]:
problem.objective.check_grad(
    x = problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
rho,-4.441204e+03,-4415.069142,-4467.290064,-4441.179603,52.220923,0.024858,5.597168e-06
kappa,3.767917e+02,402.408181,349.550105,375.979143,52.858076,0.812580,2.161238e-03
psi,-6.221777e+01,-33.480002,-90.844546,-62.162274,57.364544,0.055496,8.927586e-04
phi,-2.889060e+02,-260.384722,-317.569465,-288.977094,57.184743,0.071057,2.458915e-04
beta,-2.977135e-09,28.598364,-28.598368,-0.000002,57.196732,0.000002,3.171587e-01
alpha,-5.439484e-03,28.577706,-28.616018,-0.019156,57.193724,0.013717,7.164229e-01
delta,6.030195e+00,34.874088,-22.599848,6.137120,57.473937,0.106925,1.742259e-02
sigma_add,-1.028216e+04,-10253.380133,-10310.933673,-10282.156903,57.553540,0.001284,1.249140e-07
sigma_mul,-1.967307e+03,-1938.724641,-1995.889305,-1967.306973,57.164664,0.000047,2.393937e-08


In [29]:
gc = problem.objective.check_grad_multi_eps(
    x=problem.get_reduced_vector(startpoints[0]),
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-4.441204e+03,-4436.803972,-4445.576807,-4.441190e+03,8.772836,1.407181e-02,3.168478e-06,1.000000e-03
kappa,3.767917e+02,386.428954,367.113928,3.767714e+02,19.315026,2.028218e-02,5.383137e-05,1.000000e-03
psi,-6.221777e+01,-61.970436,-62.464942,-6.221769e+01,0.494506,8.090234e-05,1.300332e-06,1.000000e-03
phi,-2.889060e+02,-289.636982,-288.177855,-2.889074e+02,1.459127,1.381698e-03,4.782512e-06,1.000000e-03
beta,-2.977135e-09,0.002860,-0.002860,-2.823981e-09,0.005720,1.531536e-10,1.531536e-09,1.000000e-01
alpha,-5.439484e-03,-0.002018,-0.008941,-5.479473e-03,0.006923,3.998950e-05,4.230774e-04,1.000000e-01
delta,6.030195e+00,6.323437,5.738811,6.031124e+00,0.584626,9.290647e-04,1.540195e-04,1.000000e-03
sigma_add,-1.028216e+04,275701.535429,-296265.849101,-1.028216e+04,571967.384531,1.217992e-03,1.184569e-07,1.000000e-09
sigma_mul,-1.967307e+03,892.529788,-4827.143730,-1.967307e+03,5719.673518,4.473140e-05,2.273738e-08,1.000000e-07


In [30]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
rho,-4441.204461,-4436.803972,-4445.576807,-4441.190389,8.772836,0.014072,0.000003,0.001000
kappa,376.791723,386.428954,367.113928,376.771441,19.315026,0.020282,0.000054,0.001000
psi,-62.217770,-61.970436,-62.464942,-62.217689,0.494506,0.000081,0.000001,0.001000
phi,-288.906037,-289.636982,-288.177855,-288.907418,1.459127,0.001382,0.000005,0.001000
beta,-0.000000,0.002860,-0.002860,-0.000000,0.005720,0.000000,0.000000,0.100000
alpha,-0.005439,-0.002018,-0.008941,-0.005479,0.006923,0.000040,0.000423,0.100000
delta,6.030195,6.323437,5.738811,6.031124,0.584626,0.000929,0.000154,0.001000
sigma_add,-10282.155618,275701.535429,-296265.849101,-10282.156836,571967.384531,0.001218,0.000000,0.000000
sigma_mul,-1967.306926,892.529788,-4827.143730,-1967.306971,5719.673518,0.000045,0.000000,0.000000
